In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

In [3]:
data_path = 'C:\WRDSData'

In [4]:
SP500_symbol_series = pd.read_csv(data_path+'\constituents_csv.csv')['Symbol']
SP500_symbol_series.to_csv(data_path+'\SP500_symbol_list.txt', sep=' ', index=False, header=False)

In [5]:
SP500_data = pd.read_csv(data_path+'\SP500data.csv',dtype = {'gvkey':str,
                                                             'iid':str,
                                                             'datadate':str,
                                                             'tic':str})

In [5]:
res = 0
new_tics = []
for symbol in tqdm(SP500_symbol_series):
    my_sum = (SP500_data.tic == symbol).sum()
    if my_sum < 2517:
        res += 1
        #print(symbol,2517 - my_sum)
    else:
        new_tics.append(symbol)
print(res)


54


In [6]:
pd.DataFrame(new_tics)

,0
0,MMM
1,AOS
2,ABT
3,ABMD
4,ACN
...,...
446,XLNX
447,YUM
448,ZBRA
449,ZBH


In [10]:
def process_tic(tic):
    key_cols = ['tic', 'prccd', 'total_mv', 'pe', 'pct_chg']
    sub_table = SP500_data[SP500_data.tic == tic].copy()
    sub_table = sub_table.fillna(value={'trfd': 1, 'ajexdi': 1})
    sub_table = sub_table.fillna(method = 'ffill')
    sub_table['time'] = pd.to_datetime(sub_table.datadate)
    sub_table.set_index(sub_table.time, inplace = True)
    sub_table['pe'] = sub_table['prccd'] / sub_table['eps'] 
    sub_table['adj_price'] = (sub_table['prccd'] * sub_table['trfd'] / sub_table['ajexdi'])
    sub_table['pct_chg'] = 100*(sub_table['adj_price'] - sub_table['adj_price'].shift(1)) / sub_table['adj_price'].shift(1)
    sub_table['total_mv'] = sub_table['cshoc'] * sub_table['prccd']
    sub_table = sub_table[key_cols]['2010':'2019']
    sub_table.rename(columns={'prccd':'close'},inplace=True)
    sub_table.to_pickle(data_path + '\join_table_{}.pkl'.format(tic))

In [11]:
for tic in tqdm(new_tics):
    process_tic(tic)

NameError: name 'new_tics' is not defined

In [32]:
pd.read_pickle('C:\TUShareData\join_table_000902.SZ.pkl')

,ts_code,open,high,low,close,pre_close,change,pct_chg,vol,amount,turnover_rate,volume_ratio,pe,pb,dv_ratio,total_mv
time,,,,,,,,,,,,,,,,
2010-01-04,000902.SZ,3.4587,3.6731,3.4028,3.6219,3.4867,0.1352,3.8776,145007.93,110423.9033,9.2882,1.22,NaN,7.0230,NaN,2.004660e+05
2010-01-05,000902.SZ,3.5939,3.9854,3.5659,3.9621,3.6219,0.3402,9.3929,238263.53,197665.8650,15.2616,1.98,NaN,7.6828,NaN,2.193000e+05
2010-01-06,000902.SZ,3.9155,4.2698,3.8456,4.1253,3.9621,0.1632,4.1190,228317.38,199815.4293,14.6245,1.54,NaN,7.9992,NaN,2.283300e+05
2010-01-07,000902.SZ,4.1020,4.1905,3.8922,3.9668,4.1253,-0.1585,-3.8421,151486.11,131420.8721,9.7032,0.89,NaN,7.6918,NaN,2.195580e+05
2010-01-08,000902.SZ,3.9668,4.1532,3.9062,4.1532,3.9668,0.1864,4.6990,122730.48,107063.1994,7.8613,0.69,NaN,8.0534,NaN,2.298780e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-25,000902.SZ,7.9000,7.9500,7.8300,7.8800,7.9100,-0.0300,-0.3793,110304.57,86741.1060,0.9400,1.25,12.5539,1.5680,2.5381,1.027969e+06
2019-12-26,000902.SZ,7.8800,7.9300,7.8300,7.9300,7.8800,0.0500,0.6345,67591.53,53264.9050,0.5760,0.78,12.6336,1.5780,2.5221,1.034492e+06
2019-12-27,000902.SZ,7.9300,8.0200,7.8600,7.8800,7.9300,-0.0500,-0.6305,90057.00,71481.3380,0.7674,1.07,12.5539,1.5680,2.5381,1.027969e+06


In [6]:
SP500_data[SP500_data.tic == 'CNC']

,gvkey,iid,datadate,tic,ajexdi,cshoc,cshtrd,eps,prccd,prchd,prcld,prcod,trfd
1451618,145552,01,20091231,CNC,4.0,45403000.0,215591.0,2.00,21.17,21.8700,21.150,21.56,NaN
1451619,145552,01,20100104,CNC,4.0,45403000.0,329996.0,2.00,21.70,21.7500,21.280,21.40,NaN
1451620,145552,01,20100105,CNC,4.0,45403000.0,424354.0,2.00,21.01,21.8000,20.990,21.74,NaN
1451621,145552,01,20100106,CNC,4.0,45403000.0,271550.0,2.00,21.26,21.2800,20.920,20.95,NaN
1451622,145552,01,20100107,CNC,4.0,45403000.0,380063.0,2.00,21.79,21.8500,21.160,21.26,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454130,145552,01,20191224,CNC,1.0,413796000.0,978875.0,3.28,63.23,63.2606,62.560,63.00,NaN
1454131,145552,01,20191226,CNC,1.0,413796000.0,1870835.0,3.28,63.40,63.5600,62.990,63.34,NaN
1454132,145552,01,20191227,CNC,1.0,413796000.0,1622851.0,3.28,63.65,63.6700,63.210,63.43,NaN
1454133,145552,01,20191230,CNC,1.0,413796000.0,2199515.0,3.28,63.04,63.7900,62.795,63.78,NaN


In [8]:
pd.read_pickle(data_path+'\join_table_{}.pkl'.format('CNC'))

,tic,close,total_mv,pe,pct_chg
time,,,,,
2010-01-04,CNC,21.70,9.852451e+08,10.850000,NaN
2010-01-05,CNC,21.01,9.539170e+08,10.505000,NaN
2010-01-06,CNC,21.26,9.652678e+08,10.630000,NaN
2010-01-07,CNC,21.79,9.893314e+08,10.895000,NaN
2010-01-08,CNC,21.57,9.793427e+08,10.785000,NaN
...,...,...,...,...,...
2019-12-24,CNC,63.23,2.616432e+10,19.277439,NaN
2019-12-26,CNC,63.40,2.623467e+10,19.329268,NaN
2019-12-27,CNC,63.65,2.633812e+10,19.405488,NaN


In [44]:
factor_table = pd.read_csv('F-F_Research_Data_Factors_daily.CSV').iloc[:-2]
factor_table = factor_table.rename(columns = {
    'Unnamed: 0': 'time',
    'Mkt-RF': 'market_pct_chg',
    'SMB': 'SMB_pct_chg',
    'HML': 'HML_pct_chg',
})
factor_table = factor_table.set_index(pd.to_datetime(factor_table.iloc[:,0])).iloc[:, 1:4]
factor_table.to_pickle('./factor_returns/FamaFrench3_pct_ret.pkl')